In [ ]:
from pyincore import IncoreClient, FragilityService, MappingSet, Dataset, DataService, NetworkDataset
from pyincore.analyses.epnfunctionality import EpnFunctionality
from pyincore.analyses.epfdamage import EpfDamage
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability

In [ ]:
# Connect to IN-CORE Service
client = IncoreClient()
client.clear_cache()
data_service = DataService(client)

### Retreive EPN dataset

In [ ]:
epn_network_dataset = Dataset.from_data_service("62d85b399701070dbf8c65fe", data_service)
epn_network = NetworkDataset.from_dataset(epn_network_dataset)
epf_dataset = epn_network.nodes

### Electric Power Facility Damage

In [ ]:
epf_dmg = EpfDamage(client)
epf_dmg.set_input_dataset("epfs", epf_dataset)

fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping("5b47be62337d4a37b6197a3a"))
epf_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

epf_dmg.set_parameter("result_name", "hazus_epf_dmg_result")
epf_dmg.set_parameter("hazard_type", "earthquake")
epf_dmg.set_parameter("hazard_id", "5b902cb273c3371e1236b36b")
epf_dmg.set_parameter("num_cpu", 1)
# Run Analysis
epf_dmg.run_analysis()

In [ ]:
# Retrieve result dataset
epf_dmg_result = epf_dmg.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
epf_dmg_df = epf_dmg_result.get_dataframe_from_csv().head()

# Display top 5 rows of output data
epf_dmg_df.head()

### Chaining with Monte Carlo Simulation

In [ ]:
mc_ep = MonteCarloFailureProbability(client)
mc_ep.set_input_dataset("damage", epf_dmg_result)
mc_ep.set_parameter("result_name", "epf_dmg_mc")
mc_ep.set_parameter("num_cpu", 8)

num_sample = 1000
mc_ep.set_parameter("num_samples", num_sample)
mc_ep.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3", "DS_4"])
mc_ep.set_parameter("failure_state_keys", ["DS_3", "DS_4"])
mc_ep.run_analysis()

In [ ]:
epf_dmg_fs = mc_ep.get_output_dataset("sample_failure_state")
epf_dmg_fs.get_dataframe_from_csv().head()

### Chaining with EPN functionality analysis

In [ ]:
epn_func = EpnFunctionality(client)
epn_func.set_input_dataset("epn_network", epn_network_dataset)
epn_func.set_input_dataset("epf_sample_failure_state", epf_dmg_fs)

epn_func.set_parameter("result_name", "mmsa_epn_functionality")
epn_func.set_parameter("gate_station_node_list", [1, 2, 3, 4, 5, 6, 7, 8, 9])

# Run Analysis
epn_func.run_analysis()

In [ ]:
epn_func.get_output_dataset("failure_probability").get_dataframe_from_csv().head()

In [ ]:
epn_func.get_output_dataset("sample_failure_state").get_dataframe_from_csv().head()